## Old Circuit-Level Testing

In [ ]:
from utils import *
from bbq.polynomial import Polynomial
from bbq.bbq_code import BivariateBicycle
import numpy as np
import galois
from scipy.sparse import coo_matrix, hstack


### Utils

In [ ]:
def construct_sm_circuit(self, x_order : list, z_order : list) -> list:
    """Construct one cycle of the syndrome measurement circuit for the Bivariate Bicycle code.
    
    Parameters
    ----------
    x_order : list
        List of integers or 'Idle' defining the order of the CNOTs for x stabilisers.
    y_order : list
        List of integers or 'Idle' defining the order of the CNOTs for y stabilisers.
    
    Returns
    -------
    circ : list
        List of gates in one cycle of the syndrome circuit: ('CNOT', control_qubit, target_qubit, power), ('Idle', qubit), ('Meas_X', qubit), ('Meas_Z', qubit), ('Prep_X', qubit), ('Prep_Z', qubit).
    """
    if not isinstance(x_order, list):
        raise TypeError("x_order must be a list")
    if not isinstance(z_order, list):
        raise TypeError("y_order must be a list")
    for gate in x_order:
        if not (isinstance(gate, int) or gate == 'Idle'):
            raise TypeError("x_order must be an array of integers or 'Idle'")
    for gate in z_order:
        if not (isinstance(gate, int) or gate == 'Idle'):
            raise TypeError("z_order must be an array of integers or 'Idle'")
    if not x_order[0] == 'Idle':
        raise ValueError("First x_order round must be 'Idle'")
    if not z_order[-1] == 'Idle':
        raise ValueError("Last y_order round must be 'Idle'")
    for i in range(len(np.nonzero(self.hx[0])[0])):
        if i not in x_order:
            raise ValueError("x_order must contain all target qubits")
    for i in range(len(np.nonzero(self.hz[0])[0])):
        if i not in z_order:
            raise ValueError("y_order must contain all target qubits")
    if len(x_order) > len(z_order):
        z_order += ['Idle'] * (len(x_order) - len(z_order))
    elif len(z_order) > len(x_order):
        x_order += ['Idle'] * (len(z_order) - len(x_order))

    hx, hz = self.hx, self.hz
    a, b = self.a, self.b
    l, m, q = self.l, self.m, self.q
    field = self.field
    A, B = self.A, self.B
    qubits_dict, data_qubits, x_checks, z_checks = self.qubits_dict, self.data_qubits, self.x_checks, self.z_checks
    edges = self.edges

    # Construct the circuit
    circ = []
    U = np.identity(4*l*m, dtype=int)  # to verify CNOT order

    # For each time step, add the corresponding gate:
    # ('CNOT', control_qubit, target_qubit, power), ('Idle', qubit), ('Meas_X', qubit), ('Meas_Y', qubit), ('Prep_X', qubit)

    # Round 0: Prepare X checks, CNOT/Idle Z checks
    t = 0
    cnoted_data_qubits = []
    for qubit in x_checks:
        circ.append(('Prep_X', qubit))
    if z_order[t] == 'Idle':
        for qubit in z_checks:
            circ.append(('Idle', qubit))
    else:
        for target in z_checks:
            direction = z_order[t]
            control, power = edges[(target, direction)]
            U[qubits_dict[target], :] = (U[qubits_dict[target], :] + power * U[qubits_dict[control], :]) % field
            cnoted_data_qubits.append(control)
            circ.append(('CNOT', control, target, power))
    for qubit in data_qubits:
        if not (qubit in cnoted_data_qubits):
            circ.append(('Idle', qubit))

    # Round [1, (max-1)]: CNOT/Idle X checks, CNOT/Idle Z checks
    for t in range(1, len(x_order)-1):
        cnoted_data_qubits = []
        if x_order[t] == 'Idle':
            for qubit in x_checks:
                circ.append(('Idle', qubit))
        else:
            for control in x_checks:
                direction = x_order[t]
                target, power = edges[(control, direction)]
                U[qubits_dict[target], :] = (U[qubits_dict[target], :] + power * U[qubits_dict[control], :]) % field
                cnoted_data_qubits.append(target)
                circ.append(('CNOT', control, target, power))
        if z_order[t] == 'Idle':
            for qubit in z_checks:
                circ.append(('Idle', qubit))
        else:
            for target in z_checks:
                direction = z_order[t]
                control, power = edges[(target, direction)]
                U[qubits_dict[target], :] = (U[qubits_dict[target], :] + power * U[qubits_dict[control], :]) % field
                cnoted_data_qubits.append(control)
                circ.append(('CNOT', control, target, power))
        for qubit in data_qubits:
            if not (qubit in cnoted_data_qubits):
                circ.append(('Idle', qubit))

    # Round max: CNOT/Idle X checks, Measure Z checks
    t = -1
    cnoted_data_qubits = []
    for qubit in z_checks:
        circ.append(('Meas_Z', qubit))
    if x_order[t] == 'Idle':
        for qubit in x_checks:
            circ.append(('Idle', qubit))
    else:
        for control in x_checks:
            direction = x_order[t]
            target, power = edges[(control, direction)]
            U[qubits_dict[target], :] = (U[qubits_dict[target], :] + power * U[qubits_dict[control], :]) % field
            circ.append(('CNOT', control, target, power))
            cnoted_data_qubits.append(target)
    for qubit in data_qubits:
        if not (qubit in cnoted_data_qubits):
            circ.append(('Idle', qubit))
    
    # Round final: Measure X checks, Prepare Z checks
    for qubit in data_qubits:
        circ.append(('Idle', qubit))
    for qubit in x_checks:
        circ.append(('Meas_X', qubit))
    for qubit in z_checks:
        circ.append(('Prep_Z', qubit))

    # Test measurement circuit against max depth circuit
    V = np.identity(4*l*m, dtype=int)
    for t in range(len(x_order)):
        if not x_order[t] == 'Idle':
            for control in x_checks:
                direction = x_order[t]
                target, power = edges[(control, direction)]
                V[qubits_dict[target], :] = (V[qubits_dict[target], :] + power * V[qubits_dict[control], :]) % field
    for t in range(len(z_order)):
        if not z_order[t] == 'Idle':
            for target in z_checks:
                direction = z_order[t]
                control, power = edges[(target, direction)]
                V[qubits_dict[target], :] = (V[qubits_dict[target], :] + power * V[qubits_dict[control], :]) % field
    if not np.array_equal(U, V):
        raise ValueError("Syndrome circuit does not match max depth syndrome circuit, check stabiliser orders")

    return circ


In [ ]:
def construct_decoding_matrix(self, circ : list, error_rates : dict, num_cycles : int = 1) -> np.ndarray:
    """Construct decoding matrix for a given syndrome circuit.
    
    Parameters
    ----------
    circ : list
        List of gates in one cycle of the syndrome circuit: ('CNOT', control_qubit, target_qubit, power), ('Idle', qubit), ('Meas_X', qubit), ('Meas_Z', qubit), ('Prep_X', qubit), ('Prep_Z', qubit).
    error_rate : dict
        Dictionary of error rates for keys [Meas, Prep, Idle, CNOT].
    num_cycles : int
        Number of cycles to repeat the syndrome circuit. Default is 1.
    
    Returns
    -------
    hx_eff : coo_matrix
        Decoding matrix for X stabilisers.    
    short_hx_eff : coo_matrix
        Decoding matrix for X stabilisers without columns for logicals.    
    hz_eff : coo_matrix
        Decoding matrix for Z stabilisers.    
    short_hz_eff : coo_matrix
        Decoding matrix for Z stabilisers without columns for logicals.
    channel_prob_x : list
        List of probabilities for each X syndrome, i.e. each column in hx_eff.    
    channel_prob_z : list
        List of probabilities for each Z syndrome, i.e. each column in hz_eff.
    """
    if not (isinstance(error_rates, dict)):
        raise TypeError("error_rates must be a dictionary")
    for key in error_rates.keys():
        if (key not in ['Meas', 'Prep', 'Idle', 'CNOT']) or (len(error_rates) != 4):
            raise ValueError("error_rates must have keys ['Meas', 'Prep', 'Idle', 'CNOT']")
        if not (isinstance(error_rates[key], float) and 0 <= error_rates[key] <= 1):
            raise ValueError("error_rates must have values between 0 and 1")
    if not (isinstance(num_cycles, int) and num_cycles > 0):
        raise TypeError("num_cycles must be a positive integer")

    l, m = self.l, self.m
    field = self.field
    qubits_dict, data_qubits = self.qubits_dict, self.data_qubits
    x_logicals, z_logicals = self.x_logicals, self.z_logicals
    x_checks, z_checks = self.x_checks, self.z_checks

    # Construct repeated circuit
    repeated_circ = circ * num_cycles

    # Single error circuits
    z_prob, z_circuit = [], []
    x_prob, x_circuit = [], []
    head = []
    tail = repeated_circ.copy()
    for gate in repeated_circ:
        # assert gate[0] in ['CNOT', 'Idle', 'Meas_X', 'Meas_Z', 'Prep_X', 'Prep_Z']
        if gate[0] == 'Meas_X':
            # Meas_X error only affects Z detectors
            z_circuit.append(head + [('Z', gate[1])] + tail)
            z_prob.append(error_rates['Meas'])
        if gate[0] == 'Meas_Z':
            # Meas_Z error only affects X detectors
            x_circuit.append(head + [('X', gate[1])] + tail)
            x_prob.append(error_rates['Meas'])
        head.append(gate)
        tail.pop(0)
        # assert repeated_circ == head + tail
        if gate[0] == 'Prep_X':
            # Prep_X error only affects Z detectors
            z_circuit.append(head + [('Z', gate[1])] + tail)
            z_prob.append(error_rates['Prep'])
        if gate[0] == 'Prep_Z':
            # Prep_Z error only affects X detectors
            x_circuit.append(head + [('X', gate[1])] + tail)
            x_prob.append(error_rates['Prep'])
        if gate[0] == 'Idle':
            # Idle error on Z detectors
            z_circuit.append(head + [('Z', gate[1])] + tail)
            z_prob.append(error_rates['Idle']*2/3)  # 3 possible Idle errors are X, Y, Z so Z is 2/3 (Y and Z)
            # Idle error on X detectors
            x_circuit.append(head + [('X', gate[1])] + tail)
            x_prob.append(error_rates['Idle']*2/3)
        if gate[0] == 'CNOT':
            # Z error on control
            z_circuit.append(head + [('Z', gate[1])] + tail)
            z_prob.append(error_rates['CNOT']*4/15)  # possible CNOT errors are IX, IY, ..., ZZ so Z is 4/15 (IZ, IY, XZ and XY)
            # Z error on target
            z_circuit.append(head + [('Z', gate[2])] + tail)
            z_prob.append(error_rates['CNOT']*4/15)
            # Z error on both
            z_circuit.append(head + [('ZZ', gate[1], gate[2])] + tail)
            z_prob.append(error_rates['CNOT']*4/15)
            # X error on control
            x_circuit.append(head + [('X', gate[1])] + tail)
            x_prob.append(error_rates['CNOT']*4/15)
            # X error on target
            x_circuit.append(head + [('X', gate[2])] + tail)
            x_prob.append(error_rates['CNOT']*4/15)
            # X error on both
            x_circuit.append(head + [('XX', gate[1], gate[2])] + tail)
            x_prob.append(error_rates['CNOT']*4/15)

    # Execute each noisy X circuit and compute syndrome
    # Add two noiseless syndrome cycles to end
    cnt = 0
    Hx_dict = {}
    for x_circ in x_circuit:
        syndrome_history, state, syndrome_map, err_cnt = self._simulate_x_circuit(x_circ + circ + circ)
        assert err_cnt == 1
        assert len(syndrome_history) == l * m * (num_cycles + 2)

        # Compute final state of data qubits and logical effect
        state_data_qubits = [state[qubits_dict[qubit]] for qubit in data_qubits]
        syndrome_final_logical = (np.array(z_logicals) @ state_data_qubits) % field

        # Syndrome sparsification, i.e. only keep syndrome entries that change from previous cycle
        syndrome_history_copy = syndrome_history.copy()
        for check in z_checks:
            pos = syndrome_map[check]
            assert len(pos) == num_cycles + 2
            for row in range(1, num_cycles + 2):
                syndrome_history[pos[row]] += syndrome_history_copy[pos[row-1]]
        syndrome_history %= field

        # Combine syndrome_history and syndrome_final_logical
        syndrome_history_augmented = np.hstack([syndrome_history, syndrome_final_logical])

        # Hx_dict maps flagged Z stabilisers to corresponding noisy circuit, i.e. Hx_dict[flagged_z_stab] = [noisy_circuit_1, noisy_circuit_2, ...]
        supp = tuple(np.nonzero(syndrome_history_augmented)[0])
        if supp in Hx_dict:
            Hx_dict[supp].append(cnt)
        else:
            Hx_dict[supp] = [cnt]
        cnt += 1

    first_logical_row_x = l * m * (num_cycles + 2)
    num_x_errors = len(Hx_dict)  # Number of distinct X syndrome histories
    k = len(x_logicals) # Number of logical qubits
    hx_eff, short_hx_eff = [], []
    channel_prob_x = []
    for supp in Hx_dict:
        new_col = np.zeros((l * m * (num_cycles + 2) + k, 1), dtype=int)  # With the augmented part for logicals
        new_col_short = np.zeros((l * m * (num_cycles + 2), 1), dtype=int)
        new_col[list(supp), 0] = 1  # 1 indicates which stabiliser is flagged
        new_col_short[:, 0] = new_col[0:first_logical_row_x, 0]
        hx_eff.append(coo_matrix(new_col))
        short_hx_eff.append(coo_matrix(new_col_short))
        channel_prob_x.append(np.sum([x_prob[i] for i in Hx_dict[supp]]))  # Probability of a given x syndrome
    hx_eff = hstack(hx_eff)  # Column = error mechanism, row = flagged stabilisers
    short_hx_eff = hstack(short_hx_eff)  # Shortened hx_eff without columns for logicals

    # Execute each noisy Z circuit and compute syndrome
    # Add two noiseless syndrome cycles to end
    cnt = 0
    Hz_dict = {}
    for z_circ in z_circuit:
        syndrome_history, state, syndrome_map, err_cnt = self._simulate_z_circuit(z_circ + circ + circ)
        assert err_cnt == 1
        assert len(syndrome_history) == l * m * (num_cycles + 2)

        # Compute final state of data qubits and logical effect
        state_data_qubits = [state[qubits_dict[qubit]] for qubit in data_qubits]
        syndrome_final_logical = (np.array(x_logicals) @ state_data_qubits) % field

        # Syndrome sparsification, i.e. only keep syndrome entries that change from previous cycle
        syndrome_history_copy = syndrome_history.copy()
        for check in x_checks:
            pos = syndrome_map[check]
            assert len(pos) == num_cycles + 2
            for row in range(1, num_cycles + 2):
                syndrome_history[pos[row]] += syndrome_history_copy[pos[row-1]]
        syndrome_history %= field

        # Combine syndrome_history and syndrome_final_logical
        syndrome_history_augmented = np.hstack([syndrome_history, syndrome_final_logical])

        # Hz_dict maps flagged X stabilisers to corresponding noisy circuit, i.e. Hz_dict[flagged_x_stab] = [noisy_circuit_1, noisy_circuit_2, ...]
        supp = tuple(np.nonzero(syndrome_history_augmented)[0])
        if supp in Hz_dict:
            Hz_dict[supp].append(cnt)
        else:
            Hz_dict[supp] = [cnt]
        cnt += 1

    first_logical_row_z = l * m * (num_cycles + 2)
    num_z_errors = len(Hz_dict)  # Number of distinct Z syndrome histories
    hz_eff, short_hz_eff = [], []
    channel_prob_z = []
    for supp in Hz_dict:
        new_col = np.zeros((l * m * (num_cycles + 2) + k, 1), dtype=int)  # With the augmented part for logicals
        new_col_short = np.zeros((l * m * (num_cycles + 2), 1), dtype=int)
        new_col[list(supp), 0] = 1  # 1 indicates which stabiliser is flagged
        new_col_short[:, 0] = new_col[0:first_logical_row_z, 0]
        hz_eff.append(coo_matrix(new_col))
        short_hz_eff.append(coo_matrix(new_col_short))
        channel_prob_z.append(np.sum([z_prob[i] for i in Hz_dict[supp]]))  # Probability of a given z syndrome
    hz_eff = hstack(hz_eff)  # Row = flagged stabilisers, column = noisy circuit
    short_hz_eff = hstack(short_hz_eff)  # Shortened hz_eff without columns for logicals

    return hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z, x_circuit, z_circuit


In [ ]:
def rref_with_pivots(A, v, x = None):
    """
    Perform Gaussian elimination to find the reduced row echelon form (RREF).
    Also identifies the pivot columns.
    Also reduces a vector to keep a linear system invariant.
    
    Parameters
    ----------
    A : Galois field array
        Galois field matrix to row reduce
        
    Returns
    -------
    A_rref : Galois field array
        Row-reduced form of A
    pivots : list
        Indices of pivot columns
    """
    # Get a copy to avoid modifying the original
    A_rref = A.copy()
    v_rref = v.copy()
    m, n = A_rref.shape
    assert v.shape == (m,)
    # assert (A_rref @ x == v_rref).all()
    
    # Track the pivot positions
    pivot_cols = []
    pivot_rows = []
    
    # Iterate through columns
    for c in range(n):
        # Find pivot in column c
        for r in range(m):
            if A_rref[r, c] != 0 and r not in pivot_rows:
                break
        else:
            continue

        # Record this column as a pivot column
        pivot_cols.append(c)
        pivot_rows.append(r)
        
        # Scale the pivot row to make the pivot element 1
        pivot = A_rref[r, c]
        A_rref[r] = A_rref[r] / pivot
        v_rref[r] = v_rref[r] / pivot
        
        # Eliminate other elements in the pivot column
        for i in range(m):
            if i != r and A_rref[i, c] != 0:
                v_rref[i] = v_rref[i] - A_rref[i,c] * v_rref[r]
                A_rref[i] = A_rref[i] - A_rref[i, c] * A_rref[r]
        
        # If we've exhausted all rows, we're done
        if len(pivot_rows) == m:
            break
    
    # if len(pivot_rows) < A.shape[0]:
    #     print("Matrix is not full rank.")

    return A_rref[sorted(pivot_rows)], v_rref[sorted(pivot_rows)], pivot_cols


In [ ]:
def dijkstra_osd_decode(self, syndrome_history, prior, h_eff, debug = False):
    """Decode the syndrome using D+OSD.
    
    Parameters
    ----------
    syndrome_history : nd.array
        The syndrome of the error.
    prior : nd.array
        The probability of each error mechanism.
    h_eff : nd.array
        The effective parity check matrix, where columns = error mechanism and rows = syndrome (flagged stabilisers).

    Returns
    -------
    error : nd.array
        The estimated error.
    """
    if not isinstance(syndrome_history, np.ndarray):
        raise TypeError('syndrome must be a numpy array')
    if not isinstance(prior, np.ndarray):
        raise TypeError('prior must be a np.ndarray')

    field = self.field
    qubits_dict = self.qubits_dict
    data_qubits, x_checks, z_checks = self.data_qubits, self.x_checks, self.z_checks

    m, n = h_eff.shape

    # Mini-Dijkstra
    check_distances = np.ones(m) * n
    error_distances = np.ones(n) * n

    # Set the distance each detector is from an error
    for c in syndrome_history.nonzero()[0]:
        check_distances[c] = 0

    update_made = True
    while update_made:
        update_made = False
        for c in range(m):
            current_distance = check_distances[c]
            for e in np.nonzero(h_eff[c])[0]:
                if current_distance + 1 < error_distances[e]:
                    error_distances[e] = current_distance + 1
                    update_made = True

        for e in range(n):
            current_distance = error_distances[e]
            for c in np.nonzero(h_eff[:,e])[0]:
                if current_distance + 1 < check_distances[c]:
                    check_distances[c] = current_distance + 1
                    update_made = True

    certainties = error_distances

    # Sort errors by how certain we are of their values (most to least)
    col_rank_perm = np.argsort(certainties)
    col_rank_inv_perm = np.empty_like(col_rank_perm)
    col_rank_inv_perm[col_rank_perm] = np.arange(len(col_rank_perm))

    # Create Galois field elements
    GF = galois.GF(field)

    # Convert H and syndrome to Galois field arrays
    H_gf = GF(h_eff.copy())
    syndrome_gf = GF(syndrome_history.copy())
    
    # Order the columns of h_gf and prior according to the ranking
    H_ordered_gf = H_gf[:, col_rank_perm]
    priors_perm = prior[col_rank_perm]

    # Find the reduced row echelon form (RREF) and identify pivot columns
    H_rref_gf, syndrome_rref_gf, pivot_cols = rref_with_pivots(H_ordered_gf, syndrome_gf)
    m_ind = H_rref_gf.shape[0]
    non_pivot_cols = [i for i in range(n) if i not in pivot_cols]

    # Select the first rank(h_gf) linearly independent columns as basis set in P, others in B
    P = H_rref_gf[:, pivot_cols]
    assert P.shape == (m_ind, m_ind)
    B = H_rref_gf[:, non_pivot_cols]

    def sln_from(g):
        assert g.shape == (n - m_ind,)
        remainder =  syndrome_rref_gf - B @ g
        fix = np.linalg.solve(P, remainder)
        assert (P @ fix + B @ g == syndrome_rref_gf).all()

        score = 0
        sln = GF.Zeros(n)
        # Find prob of basis set
        for i in range(m_ind):
            p = priors_perm[pivot_cols[i], fix[i]]
            sln[pivot_cols[i]] = fix[i]
            if p > 0:
                score += np.log(p)
            else:
                score -= 1000
        
        for i in range(n - m_ind):
            p = priors_perm[non_pivot_cols[i], g[i]]
            sln[non_pivot_cols[i]] = g[i]
            if p > 0:
                score += np.log(p)
            else:
                score -= 1000

        # Check sln makes correct syndrome
        assert (H_rref_gf @ sln == syndrome_rref_gf).all()
        assert (H_gf @ sln[col_rank_inv_perm] == syndrome_gf).all()

        assert ((h_eff @ sln[col_rank_inv_perm].__array__()) % field == syndrome_history).all()

        return np.array(sln[col_rank_inv_perm]), score

    # OSD_0 solution
    best_solution, best_score = sln_from(GF.Zeros(n - m_ind))
    assert ((h_eff @ best_solution) % field == syndrome_history).all()
    pivot_col_labels = {col_rank_perm[c]: int(error_distances[col_rank_perm[c]]) for c in pivot_cols}

    # print(f'for debugging: {best_solution}, {True}, {[col_rank_perm[i] for i in pivot_cols]}, {pivot_col_labels}')

    return best_solution


In [ ]:
def simulate(self, iterations, circ, error_rates, num_cycles, hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z, verbose=False):
    """Simulate the code for given iterations.
    
    Parameters
    ----------
    iterations : int
        The number of iterations of the simulation.
    circ : list
        The syndrome measurement circuit.
    error_rates : dict
        Dictionary with error rates with keys ['Meas', 'Prep', 'Idle', 'CNOT'].
    num_cycles : int
        Number of cycles to repeat the syndrome circuit.
    hx_eff : nd.array
        The effective X parity check matrix.
    short_hx_eff : nd.array
        The effective X parity check matrix without logicals.
    hz_eff : nd.array
        The effective Z parity check matrix.
    short_hz_eff : nd.array
        The effective Z parity check matrix without logicals.
    channel_prob_x : list
        The probability of each X error mechanism.
    channel_prob_z : list
        The probability of each Z error mechanism.

    Returns
    -------
    float
        Logical error rate.
    """
    field = self.field
    l, m = self.l, self.m
    qubits_dict = self.qubits_dict
    data_qubits = self.data_qubits
    x_checks, z_checks = self.x_checks, self.z_checks
    x_logicals, z_logicals = self.x_logicals, self.z_logicals
    hx_eff, hz_eff = hx_eff.toarray(), hz_eff.toarray()
    short_hx_eff, short_hz_eff = short_hx_eff.toarray(), short_hz_eff.toarray()
    first_logical_row = l * m * (num_cycles + 2)
    k = len(x_logicals)

    # Set up priors
    x_prior = np.zeros((len(channel_prob_x), field), dtype=float)
    z_prior = np.zeros((len(channel_prob_z), field), dtype=float)

    for i, prob in enumerate(channel_prob_x):
        x_prior[i, 0] = 1 - prob
        for j in range(1, field):
            x_prior[i, j] = prob / (field - 1)
    for i, prob in enumerate(channel_prob_z):
        z_prior[i, 0] = 1 - prob
        for j in range(1, field):
            z_prior[i, j] = prob / (field - 1)

    # Run for given number of iterations
    success_count = 0
    for _ in range(iterations):

        # Generate noisy circuit
        noisy_circ, err_cnt = self._generate_noisy_circuit(circ * num_cycles, error_rates)
        if verbose:
            print(f'Number of errors: {err_cnt}')

        # Run X decoding and Z decosing sequentially
        x_success, z_success = False, False

        # Correct X errors
        x_syndrome_history, x_state, x_syndrome_map, x_err_count = self._simulate_x_circuit(noisy_circ + circ + circ)
        if verbose:
            print(f'Number of X errors: {x_err_count}')
            print(f'x_syndrome_history: {x_syndrome_history}')
        x_state_data_qubits = [x_state[qubits_dict[qubit]] for qubit in data_qubits]
        x_syndrome_final_logical = (np.array(z_logicals) @ x_state_data_qubits) % field
        # Syndrome sparsification
        x_syndrome_history_copy = x_syndrome_history.copy()
        for check in z_checks:
            pos = x_syndrome_map[check]
            assert len(pos) == num_cycles + 2
            for row in range(1, num_cycles + 2):
                x_syndrome_history[pos[row]] += x_syndrome_history_copy[pos[row-1]]
        x_syndrome_history %= field
        x_error = dijkstra_osd_decode(self, x_syndrome_history, x_prior, short_hx_eff)

        assert np.all((short_hx_eff @ x_error) % field == x_syndrome_history)

        if verbose:
            print(f'x_error: {x_error}')
            print(f'short_hx_eff @ x_error == syndrome: {((short_hx_eff @ x_error) % field == x_syndrome_history).all()}')

        # Check logical effect
        x_syndrome_history_augmented_guessed = (hx_eff @ x_error) % field
        x_syndrome_final_logical_guessed = x_syndrome_history_augmented_guessed[first_logical_row: first_logical_row + k]
        if verbose:
            print(f'{x_syndrome_final_logical_guessed=}')
            print(f'{x_syndrome_final_logical=}')
        x_success = np.array_equal(x_syndrome_final_logical_guessed, x_syndrome_final_logical)

        # Correct Z errors
#        if x_success:
#            z_syndrome_history, z_state, z_syndrome_map, z_err_count = self._simulate_z_circuit(noisy_circ + circ + circ)
#            # print(f'z_syndrome_history: {z_syndrome_history}')
#            z_state_data_qubits = [z_state[qubits_dict[qubit]] for qubit in data_qubits]
#            z_syndrome_final_logical = (np.array(x_logicals) @ z_state_data_qubits) % field
#            # Syndrome sparsification
#            z_syndrome_history_copy = z_syndrome_history.copy()
#            for check in x_checks:
#                pos = z_syndrome_map[check]
#                assert len(pos) == num_cycles + 2
#                for row in range(1, num_cycles + 2):
#                    z_syndrome_history[pos[row]] += z_syndrome_history_copy[pos[row-1]]
#            z_syndrome_history %= field
#            z_error = dijkstra_osd_decode(self, z_syndrome_history, z_prior, short_hz_eff)

            # Check logical effect
#            z_syndrome_history_augmented_guessed = (hz_eff @ z_error) % field
#            z_syndrome_final_logical_guessed = z_syndrome_history_augmented_guessed[first_logical_row: first_logical_row + k]
#            z_success = np.array_equal(z_syndrome_final_logical_guessed, z_syndrome_final_logical)
        z_success = True

        # Check if the decoding was successful
        success = x_success and z_success

        if success:
            success_count += 1

    return 1 - (success_count / iterations)


### Decoder

In [ ]:
def simulate(self, iterations, circ, error_rates, num_cycles, hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z, max_iter, verbose=False):
    """Simulate the code for given iterations.
    
    Parameters
    ----------
    iterations : int
        The number of iterations of the simulation.
    circ : list
        The syndrome measurement circuit.
    error_rates : dict
        Dictionary with error rates with keys ['Meas', 'Prep', 'Idle', 'CNOT'].
    num_cycles : int
        Number of cycles to repeat the syndrome circuit.
    hx_eff : nd.array
        The effective X parity check matrix.
    short_hx_eff : nd.array
        The effective X parity check matrix without logicals.
    hz_eff : nd.array
        The effective Z parity check matrix.
    short_hz_eff : nd.array
        The effective Z parity check matrix without logicals.
    channel_prob_x : list
        The probability of each X error mechanism.
    channel_prob_z : list
        The probability of each Z error mechanism.
    max_iter : int
        Number of max iterations for the BP decoder.
    verbose : bool
        If True, print debugging information.

    Returns
    -------
    float
        Logical error rate.
    """
    field = self.field
    l, m = self.l, self.m
    qubits_dict = self.qubits_dict
    data_qubits = self.data_qubits
    x_checks, z_checks = self.x_checks, self.z_checks
    x_logicals, z_logicals = self.x_logicals, self.z_logicals
    hx_eff, hz_eff = hx_eff.toarray(), hz_eff.toarray()
    short_hx_eff, short_hz_eff = short_hx_eff.toarray(), short_hz_eff.toarray()
    first_logical_row = l * m * (num_cycles + 2)
    k = len(x_logicals)

    # Set up priors
    x_prior = np.zeros((len(channel_prob_x), field), dtype=float)
    z_prior = np.zeros((len(channel_prob_z), field), dtype=float)

    for i, prob in enumerate(channel_prob_x):
        x_prior[i, 0] = 1 - prob
        for j in range(1, field):
            x_prior[i, j] = prob / (field - 1)
    for i, prob in enumerate(channel_prob_z):
        z_prior[i, 0] = 1 - prob
        for j in range(1, field):
            z_prior[i, j] = prob / (field - 1)

    # Run for given number of iterations
    success_count = 0
    for _ in range(iterations):

        # Generate noisy circuit
        noisy_circ, err_cnt = self._generate_noisy_circuit(circ * num_cycles, error_rates)
        if verbose:
            print(f'Number of errors: {err_cnt}')

        # Run X decoding and Z decosing sequentially
        x_success, z_success = False, False

        # Correct X errors
        x_syndrome_history, x_state, x_syndrome_map, x_err_count = self._simulate_x_circuit(noisy_circ + circ + circ)
        if verbose:
            print(f'Number of X errors: {x_err_count}')
            print(f'x_syndrome_history: {x_syndrome_history}')
        x_state_data_qubits = [x_state[qubits_dict[qubit]] for qubit in data_qubits]
        x_syndrome_final_logical = (np.array(z_logicals) @ x_state_data_qubits) % field
        # Syndrome sparsification
        x_syndrome_history_copy = x_syndrome_history.copy()
        for check in z_checks:
            pos = x_syndrome_map[check]
            assert len(pos) == num_cycles + 2
            for row in range(1, num_cycles + 2):
                x_syndrome_history[pos[row]] += x_syndrome_history_copy[pos[row-1]]
        x_syndrome_history %= field
        x_error, x_decoding_success = bp_osd_decode(self, x_syndrome_history, x_prior, short_hx_eff, max_iter)

        if not x_decoding_success:
            return 'Failed to decode :('

        if verbose:
            print(f'x_error: {x_error}')
            print(f'short_hx_eff @ x_error == syndrome: {((short_hx_eff @ x_error) % field == x_syndrome_history).all()}')

        assert np.all((short_hx_eff @ x_error) % field == x_syndrome_history)

        # Check logical effect
        x_syndrome_history_augmented_guessed = (hx_eff @ x_error) % field
        x_syndrome_final_logical_guessed = x_syndrome_history_augmented_guessed[first_logical_row: first_logical_row + k]
        if verbose:
            print(f'{x_syndrome_final_logical_guessed=}')
            print(f'{x_syndrome_final_logical=}')
        x_success = np.array_equal(x_syndrome_final_logical_guessed, x_syndrome_final_logical)

        # Correct Z errors
#        if x_success:
#            z_syndrome_history, z_state, z_syndrome_map, z_err_count = self._simulate_z_circuit(noisy_circ + circ + circ)
#            # print(f'z_syndrome_history: {z_syndrome_history}')
#            z_state_data_qubits = [z_state[qubits_dict[qubit]] for qubit in data_qubits]
#            z_syndrome_final_logical = (np.array(x_logicals) @ z_state_data_qubits) % field
#            # Syndrome sparsification
#            z_syndrome_history_copy = z_syndrome_history.copy()
#            for check in x_checks:
#                pos = z_syndrome_map[check]
#                assert len(pos) == num_cycles + 2
#                for row in range(1, num_cycles + 2):
#                    z_syndrome_history[pos[row]] += z_syndrome_history_copy[pos[row-1]]
#            z_syndrome_history %= field
#            z_error = dijkstra_osd_decode(self, z_syndrome_history, z_prior, short_hz_eff)

            # Check logical effect
#            z_syndrome_history_augmented_guessed = (hz_eff @ z_error) % field
#            z_syndrome_final_logical_guessed = z_syndrome_history_augmented_guessed[first_logical_row: first_logical_row + k]
#            z_success = np.array_equal(z_syndrome_final_logical_guessed, z_syndrome_final_logical)
        z_success = True

        # Check if the decoding was successful
        success = x_success and z_success

        if success:
            success_count += 1

    return 1 - (success_count / iterations)


In [ ]:
def _err_to_det(h_eff):
    """Find the detectors in the neighbourhood of each error, i.e. error : [neighbouring detectors]."""
    det, err = np.nonzero(h_eff)
    err_neighbourhood = {}
    for i in range(len(err)):
        if err[i] not in err_neighbourhood:
            err_neighbourhood[int(err[i])] = [int(det[i])]
        else:
            err_neighbourhood[int(err[i])].append(int(det[i]))
    return err_neighbourhood

def _det_to_err(h_eff):
    """Find the errors in the neighbourhood of each detector, i.e. detector : [neighbouring errors]."""
    det, err = np.nonzero(h_eff)
    det_neighourhood = {}
    for i in range(len(det)):
        if det[i] not in det_neighourhood:
            det_neighourhood[int(det[i])] = [int(err[i])]
        else:
            det_neighourhood[int(det[i])].append(int(err[i]))
    return det_neighourhood

def bp_osd_decode(self, syndrome, prior, h_eff, max_iter, bp_only=False):
    """Decode the syndrome using BP+OSD.
    
    Parameters
    ----------
    syndrome : nd.array
        The syndrome of the error.
    prior : nd.array
        The probability of each error mechanism.
    h_eff : nd.array
        The effective parity check matrix, where columns = error mechanism and rows = syndrome (flagged stabilisers).
    max_iter : int
        The maximum number of iterations.

    Returns
    -------
    error : nd.array
        The error.
    success : bool
        Whether the decoding was successful.
    """
    if not isinstance(syndrome, np.ndarray):
        raise TypeError('syndrome must be a numpy array')
    if not isinstance(prior, np.ndarray):
        raise TypeError('prior must be a numpy array')
    if not (isinstance(max_iter, int) and max_iter > 0):
        raise ValueError('max_iter must be a positive integer')

    field = self.field
    qubits_dict = self.qubits_dict
    data_qubits, x_checks, z_checks = self.data_qubits, self.x_checks, self.z_checks
    n_detectors, n_errors = h_eff.shape

    err_neighbourhood = _err_to_det(h_eff)
    det_neighbourhood = _det_to_err(h_eff)

    # ------------------------BP--------------------------------

    # Step 0: initialisation
    # Q[k, i] is the message passed from error k to check i
    Q = np.zeros((n_errors, n_detectors, field))
    for i in range(n_errors):
        # Send the same message of priors for each error to its neighbouring detectors
    
        #######################################################################
        # WARNING: If an error flags no detectors, sets messages to 0, => if syndrome is all 0, then will always say 0 errors (not a possible non-0 solution) *I think*
        #######################################################################
    
        if i in err_neighbourhood:
            dets = err_neighbourhood[i]
            Q[i, dets, :] = prior[i]
    # P[i, k] is the message passed from check i to error k
    P = np.zeros((n_detectors, n_errors, field))

    for _ in range(max_iter):
        posteriors = np.zeros(prior.shape)

        # Step 1: pass check to error messages

        for i in det_neighbourhood:
            errs = det_neighbourhood[i]

            # Isolate the relevant error messages
            convolution = Q[errs, i, :].copy()

            # Fourier transform the error messages
            convolution = np.fft.fft(convolution, axis=1)

            for j in range(len(errs)):
                # Remove the j-th error message from the convolution
                sub_convolution = np.delete(convolution, j, axis=0)

                # Compute the product of the transformed error messages
                sub_convolution = np.prod(sub_convolution, axis=0)

                # Inverse Fourier transform the product to find the convolution
                sub_convolution = np.fft.ifft(sub_convolution, axis=0)

                # Pass message
                for k in range(field):
                    P[i, errs[j], k] = sub_convolution[(syndrome[i]-k) % field]

        # Step 2: pass error to check messages

        error = np.zeros(n_errors, dtype=int)

        for i in err_neighbourhood:
            dets = err_neighbourhood[i]

            # Isolate the relevant check messages
            posterior = P[dets, i, :].copy()
            # print(f'posterior: \n {posterior}')

            for j in range(len(dets)):
                # Remove the j-th check message from the posterior
                sub_posterior = np.delete(posterior, j, axis=0)

                # Compute the product of probabilities
                sub_posterior = np.prod(sub_posterior, axis=0) * prior[i, :]

                #################################################
                # WARNING: sub_posterior is no longer normalised!
                #################################################

                # Pass message
                Q[i, dets[j], :] = sub_posterior / np.sum(sub_posterior)

            # TODO: sub_posterior out of scope

            # Step 3: calculate posterior and make hard decision on errors
            posterior = np.prod(posterior, axis=0) * prior[i, :]
            posterior_sum = np.sum(posterior)
            for j in range(len(posterior)):
                posterior[j] = posterior[j] / (posterior_sum - posterior[j])  ####### do I have blowing up problems here??? yes, yes you do...
                posteriors[i, :] = posterior  ############### does OSD want the likelihoods or the probabilities??? (I think likelihoods here)
                if posterior[j] >= 1:
                    error[i] = j

                # TODO: posteriors[i, :] does not depend on j, error[i] is overwritten
                    
                    ##############################################
                    # WARNING: may overwrite if there are 2 error types (eg X and X^2) that are likely (ONLY happens when eg likelihoods are (0, 1, 1) so doing > 1 instead of >= 1 would fix this)
                    ##############################################

        # Step 4: check convergence
        if np.all(h_eff @ error % field == syndrome):
            return error, True, True, posteriors

    if bp_only:
        return posteriors, False


    # ------------------------OSD--------------------------------

    m, n = h_eff.shape

    ##################################################################################
    # For qubits, do normal OSD (need to be careful with error powers for qudits *help*)
    ##################################################################################

    # Step 1: order the errors by likelihood
    # Row index = error mechanism, column index = power, i.e. entry for mechanism (1, ..., field-1)
    col_rank_perm = np.argsort(-np.delete(posteriors, 0, axis=1), axis=None)
    col_rank_inv_perm = np.empty_like(col_rank_perm)
    col_rank_inv_perm[col_rank_perm] = np.arange(len(col_rank_perm))

    # Step 2: create Galois field elements
    GF = galois.GF(field)
    H_gf = GF(h_eff.copy())
    syndrome_gf = GF(syndrome.copy())

    # Order the columns of h_gf and prior according to the ranking
    H_ordered_gf = H_gf[:, col_rank_perm]
    priors_perm = posteriors[col_rank_perm]

    # Find the reduced row echelon form (RREF) and identify pivot columns
    H_rref_gf, syndrome_rref_gf, pivot_cols = rref_with_pivots(H_ordered_gf, syndrome_gf)
    m_ind = H_rref_gf.shape[0]
    non_pivot_cols = [i for i in range(n) if i not in pivot_cols]

    # Select the first rank(h_gf) linearly independent columns as basis set in P, others in B
    P = H_rref_gf[:, pivot_cols]
    assert P.shape == (m_ind, m_ind)
    B = H_rref_gf[:, non_pivot_cols]

    def sln_from(g):
        assert g.shape == (n - m_ind,)
        remainder =  syndrome_rref_gf - B @ g
        fix = np.linalg.solve(P, remainder)
        assert (P @ fix + B @ g == syndrome_rref_gf).all()

        score = 0
        sln = GF.Zeros(n)
        # Find prob of basis set
        for i in range(m_ind):
            p = priors_perm[pivot_cols[i], fix[i]]
            sln[pivot_cols[i]] = fix[i]
            if p > 0:
                score += np.log(p)
            else:
                score -= 1000
        
        for i in range(n - m_ind):
            p = priors_perm[non_pivot_cols[i], g[i]]
            sln[non_pivot_cols[i]] = g[i]
            if p > 0:
                score += np.log(p)
            else:
                score -= 1000

        # Check sln makes correct syndrome
        assert (H_rref_gf @ sln == syndrome_rref_gf).all()
        assert (H_gf @ sln[col_rank_inv_perm] == syndrome_gf).all()

        assert ((h_eff @ sln[col_rank_inv_perm].__array__()) % field == syndrome).all()

        return np.array(sln[col_rank_inv_perm]), score

    # OSD_0 solution
    best_solution, best_score = sln_from(GF.Zeros(n - m_ind))
    assert ((h_eff @ best_solution) % field == syndrome).all()
    pivot_col_labels = {col_rank_perm[c]: np.delete(posteriors, 0, axis=1).reshape(-1)[col_rank_perm[c]] for c in pivot_cols}

    # print(f'for debugging: {best_solution}, {True}, {[col_rank_perm[i] for i in pivot_cols]}, {pivot_col_labels}')

    return best_solution, True, False, posteriors


### Tests

In [ ]:
# Testing...

d = 5

a = Polynomial(2, np.array([[1, 0], [-1, 0]]))
b = Polynomial(2, np.array([[1, -1], [0, 0]]))
bb = BivariateBicycle(a, b, d, d, 1)
x_order = ['Idle', 0, 3, 1, 2]
z_order = [0, 3, 1, 2, 'Idle']
p = 0.002
num_cycles = d
error_rates = {'Meas': p, 'Prep': p, 'Idle': p, 'CNOT': p}
max_iter = 100
print(bb)

qubits_dict, data_qubits, x_checks, z_checks = bb.qubits_dict, bb.data_qubits, bb.x_checks, bb.z_checks
hx, hz = bb.hx, bb.hz
lx, lz = bb.x_logicals, bb.z_logicals
circ = construct_sm_circuit(bb, x_order, z_order)
hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z, x_circuit, z_circuit = construct_decoding_matrix(bb, circ, error_rates, num_cycles)


Bivariate Bicycle code for
a(x, y) = 1x^0y^0 + 0x^0y^1 + 1x^1y^0 + 0x^1y^1
b(x, y) = 1x^0y^0 + 1x^0y^1 + 0x^1y^0 + 0x^1y^1


In [ ]:
# d = 3
# num_cycles = 3
# p = 0.002

res = simulate(bb, 50, circ, error_rates, num_cycles, hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z, max_iter, verbose=False)
res / num_cycles


C:\Users\elean\AppData\Local\Temp\ipykernel_29108\2162902609.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, errs[j], k] = sub_convolution[(syndrome[i]-k) % field]
C:\Users\elean\AppData\Local\Temp\ipykernel_29108\2162902609.py:129: RuntimeWarning: divide by zero encountered in scalar divide
  posterior[j] = posterior[j] / (posterior_sum - posterior[j])  ####### do I have blowing up problems here??? yes, yes you do...


0.006666666666666672

In [ ]:
# d = 5
# num_cycles = 5
# p = 0.002

res = simulate(bb, 10, circ, error_rates, num_cycles, hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z, max_iter, verbose = False)
res / num_cycles


C:\Users\elean\AppData\Local\Temp\ipykernel_29108\2162902609.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, errs[j], k] = sub_convolution[(syndrome[i]-k) % field]
C:\Users\elean\AppData\Local\Temp\ipykernel_29108\2162902609.py:129: RuntimeWarning: divide by zero encountered in scalar divide
  posterior[j] = posterior[j] / (posterior_sum - posterior[j])  ####### do I have blowing up problems here??? yes, yes you do...


0.0

In [ ]:
# d=3 per cycle

0.06/3


0.02

In [ ]:
# d=5 per cycle

0.12/5


0.024

In [ ]:
x_prior = np.zeros((len(channel_prob_x), 2), dtype=float)

for i, prob in enumerate(channel_prob_x):
    x_prior[i, 0] = 1 - prob
    for j in range(1, 2):
        x_prior[i, j] = prob / (2 - 1)


In [ ]:
err_neighbourhood = _err_to_det(short_hx_eff)
det_neighbourhood = _det_to_err(short_hx_eff)
